In [1]:
import optuna
import pickle
import plotly 
from src.models.hyperopt_emos import Objective

2024-06-07 05:34:34.918066: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-07 05:34:34.920471: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-07 05:34:34.952583: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-07 05:34:40.464877: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
features_names = ['wind_speed', 'press', 'kinetic', 'humid', 'geopot']

features_names_dict = {name: 1 for name in features_names}

features_names_dict['wind_speed'] = 15

objectives = ['twCRPS12']


In [3]:
#study_name = "crps_obj_CRPS_twCRPS12_MOTPE.pkl"
study_name = "crps_obj_twCRPS12_TPE.pkl"
filepath = '/net/pc200239/nobackup/users/hakvoort/hyperopt/hyperopt_emos/'

# an optuna study
study = pickle.load(open(filepath + study_name, 'rb'))

In [186]:
study2 = optuna.load_study(study_name='twcrps_obj_CRPS_twCRPS12_MOTPE', storage=f'sqlite:///{filepath}/study.db')
study2.set_metric_names(['CRPS', 'twCRPS12'])

/tmp/ipykernel_2950728/3646120845.py:2: ExperimentalWarning:

set_metric_names is experimental (supported from v3.2.0). The interface can change in the future.



In [177]:
#give hyperparameter importances

optuna.visualization.plot_param_importances(study2).show()

In [169]:
optuna.visualization.plot_pareto_front(study2)

In [187]:
df = study2.trials_dataframe()

new_column_names = {
    'params_chain_function_mean': 'cf_mean',
    'params_chain_function_std': 'cf_std',
    'params_chain_function_constant': 'cf_constant',
    'values_CRPS': 'CRPS',
    'values_twCRPS12': 'twCRPS12',
    'params_forecast_distribution': 'forecast_distribution',
    'params_distribution_1': 'distribution_1',
    'params_distribution_2': 'distribution_2',
    'params_epochs': 'epochs',
    'params_learning_rate': 'learning_rate',
    'params_batch_size': 'batch_size',
    'params_optimizer': 'optimizer'
}

df.rename(columns=new_column_names, inplace=True)

print(df.columns)

Index(['number', 'CRPS', 'twCRPS12', 'datetime_start', 'datetime_complete',
       'duration', 'batch_size', 'cf_constant', 'cf_mean', 'cf_std',
       'distribution_1', 'distribution_2', 'epochs', 'forecast_distribution',
       'learning_rate', 'optimizer', 'params_perform_batching', 'state'],
      dtype='object')


In [188]:
top_CRPS = df.sort_values('CRPS').head(5)

In [189]:
top_twCRPS12 = df.sort_values('twCRPS12').head(5)

In [190]:

pareto_front = study2.best_trials
best_runs = [trial.number for trial in pareto_front]

pareto_trials = df['number'].isin(best_runs)

my_run = df[pareto_trials]


In [191]:
params = ['cf_mean',
           'cf_std', 
           'cf_constant',
             'forecast_distribution', 
             'CRPS', 
             'twCRPS12',
             ]

In [192]:
display(top_CRPS[params])

,cf_mean,cf_std,cf_constant,forecast_distribution,CRPS,twCRPS12
73,8.145216,7.913398,0.000104,distr_mixture_linear,2.701168,0.178784
77,11.201177,7.358788,0.000272,distr_mixture_linear,2.703183,0.179080
30,-2.596332,9.524594,0.690616,distr_trunc_normal,2.706669,0.180552
1,-3.158933,0.000276,0.009416,distr_mixture_linear,2.706887,0.181077
29,-0.482392,5.689483,1.633704,distr_trunc_normal,2.707357,0.180331


In [193]:
display(top_twCRPS12[params])

,cf_mean,cf_std,cf_constant,forecast_distribution,CRPS,twCRPS12
60,10.816164,1.785932,0.000210,distr_trunc_normal,2.845130,0.176170
54,12.199493,1.329734,0.000066,distr_trunc_normal,3.031583,0.177050
50,11.881624,1.164704,0.000083,distr_trunc_normal,3.035117,0.177576
64,9.448122,0.000117,0.001009,distr_mixture,2.869211,0.177628
23,10.560683,0.414968,0.000202,distr_trunc_normal,2.999791,0.177686


In [194]:
my_run = my_run.sort_values('CRPS')
display(my_run[params])

,cf_mean,cf_std,cf_constant,forecast_distribution,CRPS,twCRPS12
73,8.145216,7.913398,0.000104,distr_mixture_linear,2.701168,0.178784
75,9.197862,2.935806,0.000895,distr_mixture_linear,2.733251,0.177762
60,10.816164,1.785932,0.000210,distr_trunc_normal,2.845130,0.176170
